In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time

from datetime import datetime as dt
from bs4 import BeautifulSoup

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
USERNAME = "postgres"
PASSWORD = "georgia1"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/Happiness"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
file2015 = "2015.csv"
file2016 = "2016.csv"
file2017 = "2017.csv"
file2018 = "2018.csv"
file2019 = "2019.csv"
df15 = pd.read_csv(file2015)
df16 = pd.read_csv(file2016)
df17 = pd.read_csv(file2017)
df18 = pd.read_csv(file2018)
df19 = pd.read_csv(file2019)

In [4]:
df15 = df15.drop(columns=['Region', 'Happiness Rank', 'Standard Error', 'Family', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual'])

In [5]:
df15.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Switzerland,7.587,1.39651,0.94143
1,Iceland,7.561,1.30232,0.94784
2,Denmark,7.527,1.32548,0.87464
3,Norway,7.522,1.45900,0.88521
4,Canada,7.427,1.32629,0.90563


In [6]:
df15 = df15.rename({'Country': 'country', 'Happiness Score': 'happiness_score', 'Economy (GDP per Capita)': 'economy_gdp', 'Health (Life Expectancy)': 'life_expectancy'}, axis=1)
df15.head()

,country,happiness_score,economy_gdp,life_expectancy
0,Switzerland,7.587,1.39651,0.94143
1,Iceland,7.561,1.30232,0.94784
2,Denmark,7.527,1.32548,0.87464
3,Norway,7.522,1.45900,0.88521
4,Canada,7.427,1.32629,0.90563


In [7]:
df15['year'] = '2015'
df15.head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Switzerland,7.587,1.39651,0.94143,2015
1,Iceland,7.561,1.30232,0.94784,2015
2,Denmark,7.527,1.32548,0.87464,2015
3,Norway,7.522,1.45900,0.88521,2015
4,Canada,7.427,1.32629,0.90563,2015


In [8]:
df15.set_index('country')

,happiness_score,economy_gdp,life_expectancy,year
country,,,,
Switzerland,7.587,1.39651,0.94143,2015
Iceland,7.561,1.30232,0.94784,2015
Denmark,7.527,1.32548,0.87464,2015
Norway,7.522,1.45900,0.88521,2015
Canada,7.427,1.32629,0.90563,2015
...,...,...,...,...
Rwanda,3.465,0.22208,0.42864,2015
Benin,3.340,0.28665,0.31910,2015
Syria,3.006,0.66320,0.72193,2015


In [9]:
df15.to_csv('df15.csv') 

In [10]:
df16 = df16.drop(columns=['Region', 'Happiness Rank', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Family', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual'])
df16.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy)
0,Denmark,7.526,1.44178,0.79504
1,Switzerland,7.509,1.52733,0.86303
2,Iceland,7.501,1.42666,0.86733
3,Norway,7.498,1.57744,0.79579
4,Finland,7.413,1.40598,0.81091


In [11]:
df16 = df16.rename({'Country': 'country', 'Happiness Score': 'happiness_score', 'Economy (GDP per Capita)': 'economy_gdp', 'Health (Life Expectancy)': 'life_expectancy'}, axis=1)
df16.head()

,country,happiness_score,economy_gdp,life_expectancy
0,Denmark,7.526,1.44178,0.79504
1,Switzerland,7.509,1.52733,0.86303
2,Iceland,7.501,1.42666,0.86733
3,Norway,7.498,1.57744,0.79579
4,Finland,7.413,1.40598,0.81091


In [12]:
df16['year'] = '2016'
df16.head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Denmark,7.526,1.44178,0.79504,2016
1,Switzerland,7.509,1.52733,0.86303,2016
2,Iceland,7.501,1.42666,0.86733,2016
3,Norway,7.498,1.57744,0.79579,2016
4,Finland,7.413,1.40598,0.81091,2016


In [13]:
df16.set_index('country')

,happiness_score,economy_gdp,life_expectancy,year
country,,,,
Denmark,7.526,1.44178,0.79504,2016
Switzerland,7.509,1.52733,0.86303,2016
Iceland,7.501,1.42666,0.86733,2016
Norway,7.498,1.57744,0.79579,2016
Finland,7.413,1.40598,0.81091,2016
...,...,...,...,...
Benin,3.484,0.39499,0.21028,2016
Afghanistan,3.360,0.38227,0.17344,2016
Togo,3.303,0.28123,0.24811,2016


In [14]:
df16.to_csv('df16.csv')

In [15]:
df17 = df17.drop(columns=['Happiness.Rank', 'Whisker.high', 'Whisker.low', 'Family', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual'])
df17.head()

,Country,Happiness.Score,Economy..GDP.per.Capita.,Health..Life.Expectancy.
0,Norway,7.537,1.616463,0.796667
1,Denmark,7.522,1.482383,0.792566
2,Iceland,7.504,1.480633,0.833552
3,Switzerland,7.494,1.564980,0.858131
4,Finland,7.469,1.443572,0.809158


In [16]:
df17 = df17.rename({'Country': 'country', 'Happiness.Score': 'happiness_score', 'Economy..GDP.per.Capita.': 'economy_gdp', 'Health..Life.Expectancy.': 'life_expectancy'}, axis=1)
df17.head()

,country,happiness_score,economy_gdp,life_expectancy
0,Norway,7.537,1.616463,0.796667
1,Denmark,7.522,1.482383,0.792566
2,Iceland,7.504,1.480633,0.833552
3,Switzerland,7.494,1.564980,0.858131
4,Finland,7.469,1.443572,0.809158


In [17]:
df17['year'] = '2017'
df17.head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Norway,7.537,1.616463,0.796667,2017
1,Denmark,7.522,1.482383,0.792566,2017
2,Iceland,7.504,1.480633,0.833552,2017
3,Switzerland,7.494,1.564980,0.858131,2017
4,Finland,7.469,1.443572,0.809158,2017


In [18]:
df17.set_index('country')

,happiness_score,economy_gdp,life_expectancy,year
country,,,,
Norway,7.537,1.616463,0.796667,2017
Denmark,7.522,1.482383,0.792566,2017
Iceland,7.504,1.480633,0.833552,2017
Switzerland,7.494,1.564980,0.858131,2017
Finland,7.469,1.443572,0.809158,2017
...,...,...,...,...
Rwanda,3.471,0.368746,0.326425,2017
Syria,3.462,0.777153,0.500533,2017
Tanzania,3.349,0.511136,0.364509,2017


In [19]:
df17.to_csv('df17.csv')

In [20]:
df18 = df18.drop(columns=['Overall rank', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'])
df18.head()

,Country or region,Score,GDP per capita,Healthy life expectancy
0,Finland,7.632,1.305,0.874
1,Norway,7.594,1.456,0.861
2,Denmark,7.555,1.351,0.868
3,Iceland,7.495,1.343,0.914
4,Switzerland,7.487,1.420,0.927


In [21]:
df18 = df18.rename({'Country or region': 'country', 'Score': 'happiness_score', 'GDP per capita': 'economy_gdp', 'Healthy life expectancy': 'life_expectancy'}, axis=1)
df18.head()

,country,happiness_score,economy_gdp,life_expectancy
0,Finland,7.632,1.305,0.874
1,Norway,7.594,1.456,0.861
2,Denmark,7.555,1.351,0.868
3,Iceland,7.495,1.343,0.914
4,Switzerland,7.487,1.420,0.927


In [22]:
df18['year'] = '2018'
df18.head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Finland,7.632,1.305,0.874,2018
1,Norway,7.594,1.456,0.861,2018
2,Denmark,7.555,1.351,0.868,2018
3,Iceland,7.495,1.343,0.914,2018
4,Switzerland,7.487,1.420,0.927,2018


In [23]:
df18.set_index('country')

,happiness_score,economy_gdp,life_expectancy,year
country,,,,
Finland,7.632,1.305,0.874,2018
Norway,7.594,1.456,0.861,2018
Denmark,7.555,1.351,0.868,2018
Iceland,7.495,1.343,0.914,2018
Switzerland,7.487,1.420,0.927,2018
...,...,...,...,...
Yemen,3.355,0.442,0.343,2018
Tanzania,3.303,0.455,0.381,2018
South Sudan,3.254,0.337,0.177,2018


In [24]:
df18.to_csv('df18.csv')

In [25]:
df19 = df19.drop(columns=['Overall rank', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'])
df19.head()

,Country or region,Score,GDP per capita,Healthy life expectancy
0,Finland,7.769,1.340,0.986
1,Denmark,7.600,1.383,0.996
2,Norway,7.554,1.488,1.028
3,Iceland,7.494,1.380,1.026
4,Netherlands,7.488,1.396,0.999


In [26]:
df19 = df19.rename({'Country or region': 'country', 'Score': 'happiness_score', 'GDP per capita': 'economy_gdp', 'Healthy life expectancy': 'life_expectancy'}, axis=1)
df19.head()

,country,happiness_score,economy_gdp,life_expectancy
0,Finland,7.769,1.340,0.986
1,Denmark,7.600,1.383,0.996
2,Norway,7.554,1.488,1.028
3,Iceland,7.494,1.380,1.026
4,Netherlands,7.488,1.396,0.999


In [27]:
df19['year'] = '2019'
df19.head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Finland,7.769,1.340,0.986,2019
1,Denmark,7.600,1.383,0.996,2019
2,Norway,7.554,1.488,1.028,2019
3,Iceland,7.494,1.380,1.026,2019
4,Netherlands,7.488,1.396,0.999,2019


In [28]:
df19.set_index('country')

,happiness_score,economy_gdp,life_expectancy,year
country,,,,
Finland,7.769,1.340,0.986,2019
Denmark,7.600,1.383,0.996,2019
Norway,7.554,1.488,1.028,2019
Iceland,7.494,1.380,1.026,2019
Netherlands,7.488,1.396,0.999,2019
...,...,...,...,...
Rwanda,3.334,0.359,0.614,2019
Tanzania,3.231,0.476,0.499,2019
Afghanistan,3.203,0.350,0.361,2019


In [29]:
df19.to_csv('df19.csv')

In [30]:
alcohol_cons = "HappinessAlcoholConsumption.csv"
alcohol_df = pd.read_csv(alcohol_cons)
alcohol_df.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [31]:
alcohol_df = alcohol_df.drop(columns=['Region', 'Hemisphere', 'HappinessScore', 'HDI', 'GDP_PerCapita'])

In [46]:
alcohol_df = alcohol_df.rename({'Country': 'country', 'Beer_PerCapita': 'beer_percapita', 'Spirit_PerCapita': 'spirit_percapita', 'Wine_PerCapita': 'wine_percapita'}, axis=1)
alcohol_df.head()

,country,beer_percapita,spirit_percapita,wine_percapita
0,Denmark,224,81,278
1,Switzerland,185,100,280
2,Iceland,233,61,78
3,Norway,169,71,129
4,Finland,263,133,97


In [47]:
alcohol_df.to_csv('alcohol_df.csv') 

In [34]:
engine.table_names()

['2015', '2016', '2017', '2018', '2019', 'Alcohol']

In [35]:
df15.to_sql(name='2015', con=engine, if_exists='append', index=False)

In [36]:
pd.read_sql_query('select * from "2015" ', con=engine).head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Switzerland,7.587,1.39651,0.94143,2015
1,Iceland,7.561,1.30232,0.94784,2015
2,Denmark,7.527,1.32548,0.87464,2015
3,Norway,7.522,1.45900,0.88521,2015
4,Canada,7.427,1.32629,0.90563,2015


In [37]:
df16.to_sql(name='2016', con=engine, if_exists='append', index=False)

In [38]:
pd.read_sql_query('select * from "2016" ', con=engine).head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Denmark,7.526,1.44178,0.79504,2016
1,Switzerland,7.509,1.52733,0.86303,2016
2,Iceland,7.501,1.42666,0.86733,2016
3,Norway,7.498,1.57744,0.79579,2016
4,Finland,7.413,1.40598,0.81091,2016


In [39]:
df17.to_sql(name='2017', con=engine, if_exists='append', index=False)

In [40]:
pd.read_sql_query('select * from "2017" ', con=engine).head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Norway,7.537,1.616463,0.796667,2017
1,Denmark,7.522,1.482383,0.792566,2017
2,Iceland,7.504,1.480633,0.833552,2017
3,Switzerland,7.494,1.564980,0.858131,2017
4,Finland,7.469,1.443572,0.809158,2017


In [41]:
df18.to_sql(name='2018', con=engine, if_exists='append', index=False)

In [42]:
pd.read_sql_query('select * from "2018" ', con=engine).head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Finland,7.632,1.305,0.874,2018
1,Norway,7.594,1.456,0.861,2018
2,Denmark,7.555,1.351,0.868,2018
3,Iceland,7.495,1.343,0.914,2018
4,Switzerland,7.487,1.420,0.927,2018


In [43]:
df19.to_sql(name='2019', con=engine, if_exists='append', index=False)

In [44]:
pd.read_sql_query('select * from "2019" ', con=engine).head()

,country,happiness_score,economy_gdp,life_expectancy,year
0,Finland,7.769,1.340,0.986,2019
1,Denmark,7.600,1.383,0.996,2019
2,Norway,7.554,1.488,1.028,2019
3,Iceland,7.494,1.380,1.026,2019
4,Netherlands,7.488,1.396,0.999,2019


In [48]:
alcohol_df.to_sql(name='Alcohol', con=engine, if_exists='append', index=False)

In [49]:
pd.read_sql_query('select * from "Alcohol" ', con=engine).head()

,country,beer_percapita,spirit_percapita,wine_percapita
0,Denmark,224.0,81.0,278.0
1,Switzerland,185.0,100.0,280.0
2,Iceland,233.0,61.0,78.0
3,Norway,169.0,71.0,129.0
4,Finland,263.0,133.0,97.0
